In [236]:
from astropy import units as u
from astropy import coordinates as coord
import numpy as np

# Instructions
Do a cmd-F search for # UPDATE tags and enter values there. Then run the notebook. Results are printed below cells.

# Concept
After initially making this way too complicated, this is how I ultimately approached this problem.

Imagine the line between the target and contaminant as a stick that can be moved and rotated. Think of the aperture as a rectangular box. You are free to move the stick and rotate this stick in order to fit it in the box. This is effectively the same as moving and rotating the aperture, but I find it easier to visualize.

I find it helpful to consider trying to fit sticks of decreasing length. First, if the stick is longer than the diagonal, it cannot fit in the box at any angle. If it is shorter than this, it can always fit along the diagonal, so this is a good place to start. Now, if the stick is a little longer than the long dimension of the box, I could begin twisting it so that it is more aligned with the box's length and eventually the stick will contact the ends of the box. At this point I could slide the stick left and right so that one end is in a corner, but that wouldn't allow me to twist the stick any further. The same is true if I were to twist the stick, starting from the diagonal, in the other direction. So I might as well keep it centered and use that to determine my angles. 

Since I cannot increase the angles in which the stick fits by moving it, I can just consider keeping it centered. In this case, twisting the stick would cause the ends to draw out a circle with a radius equal to half the length of the stick and the contacts would be where this circle intersects the edges of the box.

(I just realized I could generalize this to any shape by starting with the stick aligned with the greatest distance between to vertices... kinda satisfying, but I won't go down that path.)

I could keep shortening the stick, but the result will be the same -- I'can get it any more tilted by sliding it around, even if it is shorter than the long dimension so that it only contacts the long sides of the box. And of course if I kept shortening it, eventually it will fit at any angle.

Now what if one end of the stick is not allowed to move beyond a certain distance from center? This is equivalent to imposing a certain accuracy on target centering in the aperture. In this case, it is still best to start with the stick aligned to a diagonal, with the stick as close to centered as possible so that the "target end" is at the maximum radius away from the center as allowable. Then I will fit the most angles by rotating the stick about the center of the box. Moving it any further off center would either violate the accuracy criterion in one direction or just cause the stick to contact a side earlier in the other direction. In this case, the problem is reduced to asking where a circle with a radius equal l/2 + (l/2 - r) = l - r, where l is the length of the stick and r is the accuracy criterion, contacts the sides of the box. QED.

In [237]:
def stick_fit_intervals(stick_length, box_width, box_length):
    # returns angles relative to the long dimension of the box that will allow the stick to fit inside
    if box_width > box_length:
        raise ValueError("Width must be less than length.")
    s, w, l = stick_length, box_width, box_length
    diag = np.sqrt(w**2 + l**2)
    # extreme cases
    if s >= diag:
        return np.empty((0,2)) * u.deg
    elif s <= w:
        return [[0, 360]] * u.deg
    
    # intermediate cases
    if s > l:
        ratio = l/s
        if hasattr(ratio, 'unit'):
            ratio = ratio.to_value('')
        min_tilt = np.arccos(ratio) * u.rad
    else: # stick fits longways
        min_tilt = 0 * u.deg
    if s > w:
        ratio = w/s
        if hasattr(ratio, 'unit'):
            ratio = ratio.to_value('')
        max_tilt = 90 * u.deg - np.arccos(ratio) * u.rad
    else:
        max_tilt = 90 * u.deg
        
    # there will be four intervals of rotation where the stick will fit for the intermediate cases
    i1 = u.Quantity((min_tilt, max_tilt))
    i2 = 180 * u.deg - i1[::-1]
    i3 = i1 - 180 * u.deg
    i4 = -i1[::-1]
    intervals = np.vstack((i1, i2, i3, i4))
    return intervals.to('deg')

Optional tests, used to validate correct output

In [238]:
print("should give empty set: ")
print(stick_fit_intervals(3, 1, 2)) 
print("should give 0, 360")
print(stick_fit_intervals(0.5, 1, 2))
print("should give a narrow range near 37 deg (3,4,5 triangle) and then other quadrants")
print(stick_fit_intervals(4.9, 3, 4)) 
print("should give 0 to a somewhat more than 37 deg + other quadrants")
print(stick_fit_intervals(3.9, 3, 4)) 
print("should give 0 to near 90 deg + other quadrants")
print(stick_fit_intervals(3.001, 3, 4)) 

should give empty set: 
[] deg
should give 0, 360
[[  0. 360.]] deg
should give a narrow range near 37 deg (3,4,5 triangle) and then other quadrants
[[  35.28126183   37.75200131]
 [ 142.24799869  144.71873817]
 [-144.71873817 -142.24799869]
 [ -37.75200131  -35.28126183]] deg
should give 0 to a somewhat more than 37 deg + other quadrants
[[   0.           50.28486277]
 [ 129.71513723  180.        ]
 [-180.         -129.71513723]
 [ -50.28486277   -0.        ]] deg
should give 0 to near 90 deg + other quadrants
[[   0.           88.52083476]
 [  91.47916524  180.        ]
 [-180.          -91.47916524]
 [ -88.52083476   -0.        ]] deg


In [239]:
def angles_fitting_two_stars(star_separation, aperture_width, aperture_height, centering_accuracy):
    """Angles relative to the axis defined by the two stars by which 
    the long axis (height) of the aperture must be rotated counterclockwise 
    in order to fit both stars."""
    if centering_accuracy > star_separation/2:
        l = star_separation
    else:
        # this allows the function to use the same stick fitting result, see concept discussion
        # it reduces to the stick length being the star separation if the target cannot be 
        # centered with an accuracy better than separation/2
        l = 2*(star_separation - centering_accuracy) 
    return stick_fit_intervals(l, aperture_width, aperture_height)

Optional tests, used to validate correct output

In [240]:
print("should give 0 to a somewhat more than 37 deg, then subsequent ranges should narrow until they disappear")
print("note that only the first range is being printed")
print(angles_fitting_two_stars(3.9, 3, 4, 5)[0])
print(angles_fitting_two_stars(3.9, 3, 4, 3.9/2*0.9)[0])
print(angles_fitting_two_stars(3.9, 3, 4, 3.9/2*0.72)[0])
print(angles_fitting_two_stars(3.9, 3, 4, 3.9/2*0.71))
print(angles_fitting_two_stars(3.9, 3, 4, 0))

should give 0 to a somewhat more than 37 deg, then subsequent ranges should narrow until they disappear
note that only the first range is being printed
[ 0.         50.28486277] deg
[21.18777598 44.3709259 ] deg
[36.74729578 36.93879385] deg
[] deg
[] deg


### Rotations and a utility

In [241]:
def account_for_contaminant_PA(pa, intervals):
    new_intervals = intervals + pa
    return new_intervals

def ORIENT_intervals(beta_y, intervals):
    orients = intervals - beta_y
    return orients.wrap_at(360*u.deg)

def merge_intervals(intervals):
    sorted_intervals = sorted(intervals, key=lambda x: x[0])
    merged = []

    for interval in sorted_intervals:
        if not merged:
            merged.append(interval)
        if (merged[-1][1] > interval[0]) or np.isclose(merged[-1][1], interval[0]):
            merged[-1][1] = max(merged[-1][1], interval[1])
        else:
            merged.append(interval)

    return merged

# Boilerplate 

In [242]:
def do_the_works(sep, pa, width, height, acc, beta_y):
    intervals = angles_fitting_two_stars(sep, width, height, acc)
    if len(intervals) == 0:
        print("Contaminant is far enough to be out of aperture for any ORIENT.")
    elif intervals[0,1] - intervals[0,0] == 360*u.deg:
        print("Contaminant is close enough to be in aperture for any ORIENT.")
    else:
        print("Contimanant could end up in the aperture for the following ORIENTS:")
        intervals = account_for_contaminant_PA(pa, intervals)
        intervals = ORIENT_intervals(beta_y, intervals)
        intervals = merge_intervals(intervals)
        intervals = u.Quantity(intervals)
        intervals = intervals.to_value('deg')
        print(intervals)
        print("Enter these into the APT to ensure contaminating source is avoided.")
        assert len(intervals) == 2
        # get the complementary ranges which will keep the target out
        buffer = 0.1
        intervals = [[intervals[1,1] + buffer, intervals[0,0] - buffer],
                     [intervals[0,1] + buffer, intervals[1,0] - buffer]]
        print(intervals)
        

# Target and contaminant coordinates

In [243]:
# can usually use Gaia DR3 coordinates. might want to think about proper motions if they are very large
target = coord.SkyCoord("12.546956711673715 -83.74376291513069", # UPDATE
                        unit=(u.deg, u.deg)) # can also use u.hourangle as needed # UPDATE
contam = coord.SkyCoord("12.541325853053971 -83.74364554629558", # UPDATE
                        unit=(u.deg, u.deg)) # UPDATE

### calculate separation and position angle 

In [244]:
sep = target.separation(contam)

# position angle (ccw from celestial north)
# remember east points left on the sky :) 
pa = coord.position_angle(target.ra, target.dec, contam.ra, contam.dec)

print(f"Contaminant is {sep.to('arcsec'):.2g} away at a position angle of {pa.to('deg'):.1f}.")

Contaminant is 2.2 arcsec away at a position angle of 280.8 deg.


### Pole Check
If either target is within striking distance of a celestial pole, angles will get weird and this notebook cannot handle them. If this cell has raised an error, then this is the case.

In [245]:
for x in (target, contam):
    if 90*u.deg - abs(x.dec) < 10*u.arcmin: # chosen to be about 10x the 52" slit height
        raise NotImplementedError("Target is too close to the pole!!!")

# Beta_y
This parameter indicating the orientation of the detector relative to Hubbles axis system seems to be either 135.0000 or 135.0559 deg for all ACQ and G140M apertures of interest. However, if this notebook is extended to new apertures, this will need to be updated.

A catalog of values is avaiable at https://www.stsci.edu/hst/instrumentation/focus-and-pointing/fov-geometry

In [246]:
beta_y = 135.000*u.deg # UDPATE?

# ACQ
For an orientation constraint to be helpful for an ACQ, the contaminating source must be between 4.5 and 7.07" from the target, otherwise it can be in the aperture for any orientation or no orientations. 

In [247]:
width = 5*u.arcsec
height = 5*u.arcsec
centering_accuracy = 2*u.arcsec
do_the_works(sep, pa, width, height, centering_accuracy, beta_y)

Contaminant is close enough to be in aperture for any ORIENT.


# G140M

In [248]:
width = 0.2*u.arcsec # UPDATE
height = 52*u.arcsec 
centering_accuracy = 52*u.arcsec # this effectively allows for any error in target position
do_the_works(sep, pa, width, height, centering_accuracy, beta_y)

Contimanant could end up in the aperture for the following ORIENTS:
[[140.72367102 150.92709563]
 [320.72367102 330.92709563]] deg
!!! these ARE NOT what you should enter into the APT.
Instead you should enter the ranges between these intervals for the orient constraints. 
Narrow the allowable ranges by 0.1 deg as a buffer against beta_y variations.
